<h3>Projects funded from USA (NSF)</h3>

<p>The National Science Foundation (NSF) funds research and education in science and engineering, through grants, contracts, and cooperative agreements. The Foundation accounts for about 20 percent of federal support to academic institutions for basic research. 

Information about research projects that NSF has funded since 1989 can be found by searching the Award Abstracts database (https://www.nsf.gov/awardsearch/download.jsp). The information includes abstracts that describe the research, and names of principal investigators and their institutions. The database includes both completed and in-process research.</p>

<p>We will use all the provided information from 1994 till 2017 (259106 awards). The dataset consists of .zip folders for each year, with .xml files for each award.</p>

In [1]:
import os
import zipfile
import cjson
import pandas as pd
import xmltodict, json

from xml.dom.minidom import parse

filenames = os.listdir('USA_data')

<p>First, we unzip each folder and then for each file of the folder, we convert the xml file to a dictionary with:
<ul>
<li>key: the file name/award id (ex. 9601223)</li>
<li>value: the actual content of the xml file in JSON format</li>
</ul></p>

In [2]:
raw_text = {}

for zipfilename in filenames:
    with zipfile.ZipFile('USA_data/'+zipfilename) as z:
        for filename in z.namelist():
            if not os.path.isdir(filename):
                try:
                    with z.open(filename) as f:
                        raw_text[filename[:-4]] = json.dumps(xmltodict.parse(f))
                except:
                    print filename
                    pass

<p>For each award/key of our dictionary, we create a new dictionary to keep the values that we'll use later on the analysis:
<ul>
<li>id: the award id</li>
<li>title: the award title</li>
<li>objective: the abstract narration of the award</li>
<li>state: the code of the state</li>
<li>subjects: information provided from 'Program Reference' and 'Program Element' values</li>
<li>foa: FoaInformation</li>
</ul>
<br>
Then, we append each dictionary to a list.
</p>

In [3]:
usa_list = []

for key, val in raw_text.items():
    usa_dict = {}

    value = cjson.decode(val)
    new_val = value['rootTag']['Award']

    usa_dict['id'] = new_val['AwardID']
    usa_dict['title'] = new_val['AwardTitle']
    usa_dict['objective'] = new_val['AbstractNarration']
    usa_dict['state'] = new_val['Institution']['StateCode']
    
    try:
        if type(new_val['ProgramReference']) is dict:
            usa_dict['subjects'] = new_val['ProgramElement']['Text'] + new_val['ProgramReference']['Text']
        elif type(new_val['ProgramReference']) is list:
            text = ''
            for i in new_val['ProgramReference']:
                text = text + ' ' + i['Text']

            usa_dict['subjects'] = text
        else:
            pass
    except:
        pass


    try:
        if type(new_val['FoaInformation']) is dict:
            usa_dict['foa'] = new_val['FoaInformation']['Name']
        elif type(new_val['FoaInformation']) is list:
            text = ''
            for i in new_val['FoaInformation']:
                text = text + ' ' + i['Name']

            usa_dict['foa'] = text
        else:
            pass
    except:
        pass

    usa_list.append(usa_dict)

<p>Finally, we use this list to create a Dataframe:<p>

In [4]:
dfUSA = pd.DataFrame(usa_list)
dfUSA

,foa,id,objective,state,subjects,title
0,NaN,9908343,9908343<br/>Ward<br/><br/>This project entitle...,MN,NaN,Directed Templating and Synthesis of Low-Densi...
1,NaN,1231468,Intellectual Merit: There is much to be learne...,NH,NaN,Using Next Generation Sequencing to Quantify t...
2,NaN,1039870,With this award from the Major Research Instru...,PA,MAJOR RESEARCH INSTRUMENTATION CHEMICAL INSTR...,MRI: Acquisition of a New 500 MHz NMR Console ...
3,NaN,1231467,This award focuses on providing support to ena...,MI,NaN,International Medical Geography Symposium; Eas...
4,NaN,1231461,Objective: <br/>The objective of this program ...,PA,Wireless comm & sig processing Computat syste...,Collaborative Research: CCSS: Cyber-Enabled Sm...
5,NaN,1042276,This project will conduct innovative research ...,PA,Cyber-Human Systems (CHS)EAGER,EAGER: Creating a Book Citation Index
6,NaN,1014801,This project synthesizes and advances understa...,DC,NaN,The Impact of Copyright Policy on Innovation i...
7,NaN,1018691,Seamless understanding of the meaning of visua...,NM,NaN,Collaborative Research: RI: Small: A Scalable ...
8,NaN,1046036,This Small Business Innovation Research (SBIR)...,PA,NaN,SBIR Phase I: Artificial Intelligence Software...
9,Other Applications NEC,0520468,Proposal Number: DEB-0520468<br/>Proposal Titl...,PA,NaN,EID: Parasite Induced Susceptibility and Trans...
